In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

import sys
sys.path.append('..')
from student_utils import adjacency_matrix_to_graph
from solver_utils import parse_input
from visualize import plot_graph

%matplotlib inline

In [ ]:
solvers = {
    'MST+DFS': mst_dfs_solve,
}

In [ ]:
"""
Solve Interface
Input:
    list_of_locations: A list of locations such that node i of the graph corresponds to name at index i of the list
    list_of_homes: A list of homes
    starting_car_location: The name of the starting location for the car
    adjacency_matrix: The adjacency matrix from the input file
Output:
    A list of locations representing the car path
    A dictionary mapping drop-off location to a list of homes of TAs that got off at that particular location
    NOTE: both outputs should be in terms of indices not the names of the locations themselves
"""

## Solver 1: MST + DFS

Idea: similar to the 2x approximation algorithm for MetricTSP, but in this case, we may not be able to remove every duplicate vertex seen (after its first occurence), because the original graph is not necessarily fully connected. Thus, we can only skip vertices when it is possible (i.e. there is an edge to take the shortcut). In this tour, we will drop everyone off at their house. No TA needs to walk.

Graphs that this is bad for:
- Branching graphs (lots of driving back and forth, where walking would be more optimal

Graphs that this is good for:
- Very connected graphs

In [ ]:
def mst_dfs_solve(list_of_locations,
                  list_of_homes,
                  starting_car_location,
                  adjacency_matrix,
                  params=[]):
    G, _ = adjacency_matrix_to_graph(adjacency_matrix)
    
    mst = nx.minimum_spanning_tree(G)
    seen = set()
    traversal = []
    def dfs(u):
        if u not in seen:
            traversal.append(u)
            seen.add(u)
            for v in mst.neighbors(u):
                if v not in seen:
                    dfs(v)
                    traversal.append(u)
    dfs(list_of_locations.index(starting_car_location))
    
    dropoffs = {
        home: home for home in list_of_homes 
    }
    return traversal, dropoffs, mst

In [ ]:
input_path = './test_inputs/random_6v_3h.in'
(num_locations,
 num_houses,
 location_names,
 house_names,
 source,
 adj) = parse_input(input_path)

G, _ = adjacency_matrix_to_graph(adj)
traversal, dropoffs = mst_dfs_solve(location_names, house_names, source, adj)
print(traversal)

In [ ]:
edges_to_draw = []
for i in range(len(traversal) - 1):
    u, v = traversal[i], traversal[i + 1]
    edges_to_draw.append((u, v))

plot_graph(input_path,
           layout_style=nx.kamada_kawai_layout,
           show_edge_weights=True,
           edges_to_draw=edges_to_draw,
           directed=True)

In [ ]:
input_path = './test_inputs/branching_10v_5h.in'

## Solver 2: Dijkstra's at each vertex

In [ ]:
def dijkstra_greedy_solve(list_of_locations,
                          list_of_homes,
                          starting_car_location,
                          adjacency_matrix,
                          params=[]):
    G, _ = adjacency_matrix_to_graph(adjacency_matrix)
    remaining = set([list_of_locations.index(name) for name in list_of_homes])
    source = curr = list_of_locations.index(starting_car_location)
    
    path = []
    
    # Continue looping until no more TAs to drop off
    while remaining:
        print(f"\n==== AT NODE #{curr} ====")
        path.append(curr)
        
        if curr in remaining:
            print(f"DROPPED TA OFF AT {curr}")
            remaining.remove(curr)

        # Which direction should we move (if at all)?
        heuristics = {}
        for n in G.neighbors(curr):
            print(n)
            distances, paths = nx.single_source_dijkstra(G, n)
            heuristics[n] = 0
            for h in remaining:
#                 if n == h:
#                     heuristics[n] += 2.0 # TODO: What should the heuristic be if you're at a house?
#                                          # Maybe just define as 1 / dist + 1
#                 else:
#                     heuristics[n] += 1 / distances[h]
                heuristics[n] += (1 / (distances[h] + 1)) ** 2 # 1/(dist+1)^2 heavily incentives closer houses
            print(f"DISTANCES FROM {n}", distances)
        
        print("HEURISTICS: ", heuristics)
        best_neighbor = max(list(heuristics.keys()), key=heuristics.get)
        print("BEST NEIGHBOR: ", best_neighbor)
        curr = best_neighbor

    if curr != source:
        sp = nx.shortest_path(G, source=curr, target=source, weight='weight')
        path += sp

    return path

In [ ]:
# input_path = './test_inputs/random_6v_3h.in'
input_path = '../phase1/50.in'
(num_locations,
 num_houses,
 location_names,
 house_names,
 source,
 adj) = parse_input(input_path)
G, _ = adjacency_matrix_to_graph(adj)
path = dijkstra_greedy_solve(location_names, house_names, source, adj)

In [ ]:
def draw_path(G, path):
    edges_to_draw = []
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        edges_to_draw.append((u, v))
    plot_graph(input_path,
               layout_style=nx.kamada_kawai_layout,
               show_edge_weights=True,
               edges_to_draw=edges_to_draw,
               directed=True)

print("PATH: ", path)
draw_path(G, path)

In [ ]:
plot_graph(input_path,
           layout_style=nx.kamada_kawai_layout,
           show_edge_weights=True,
#            edges_to_draw=edges_to_draw,
           directed=True)